In [5]:
#Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv  

# Load the image
image = cv.imread('../images/the_berry_farms_sunflower_field.jpeg', cv.IMREAD_REDUCED_COLOR_4)
gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
assert gray_image is not None, 'ERROR!!! Image not found'

# Define a function to create a Gaussian kernel
def GaussianKernal(size, sigma, show_plot=False):
    oneDKernal = np.linspace(-(size // 2), size // 2, size)
    oneDKernal = np.exp(-(oneDKernal**2) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
    twoDKernal = np.outer(oneDKernal.T, oneDKernal.T)
    twoDKernal *= 1.0 / twoDKernal.max()

    if show_plot:
        plt.imshow(twoDKernal, interpolation='none', cmap='gray')
        plt.title("Gaussian Kernel")
        plt.show()

    return twoDKernal

sigma_value = 0.64

# Create a Gaussian kernel
gaussian_filter = GaussianKernal(5, sigma_value, show_plot=False)
L_GKernal = cv.Laplacian(gaussian_filter, cv.CV_64F) * sigma_value**2           # Get the Laplacian of the Gaussian kernel
L_Gimage = cv.filter2D(gray_image, -1, L_GKernal)                               # Applying Laplacian of Gaussian kernel to the image
Threshold = 81                                                                  # Setting a Thresholdi(Adjustable)
_, binary_image = cv.threshold(np.abs(L_Gimage), Threshold, 255, cv.THRESH_BINARY)

# Get Contours
contours, _ = cv.findContours(binary_image.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for i in contours:                                                        # Mark Circles in the original image
    ((x, y), Radius) = cv.minEnclosingCircle(i)                           # Getting the center and radius of the blob
    center_coordinates = (int(x), int(y))
    Radius = int(Radius)
    cv.circle(image, center_coordinates, Radius, (255, 0, 255), 2)        # magenta coloured circles with 2 thickness

image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)                          # Convert BGR image to RGB for Matplotlib display

# Display Results
plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
plt.imshow(L_Gimage, interpolation='none', cmap='gray')
plt.title("Laplacian of Gaussian")
plt.subplot(1, 3, 2)
plt.imshow(binary_image, interpolation='none', cmap='gray')
plt.title("Binary Image")
plt.subplot(1, 3, 3)
plt.imshow(image_rgb, interpolation='none')                               # No cmap needed for RGB
plt.title("Original Image with Blobs")
plt.show()

print("Detected Number of Blobs: {}".format(len(contours)))

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
